In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import gensim
import time
import random
import snowballstemmer
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score


In [2]:
def readIMDB(path, seg='train'):
    pos_or_neg = ['pos', 'neg']
    data = []
    for label in pos_or_neg:
        files = os.listdir(os.path.join(path, seg, label))
        for file in files:
            with open(os.path.join(path, seg, label, file), 'r', encoding='utf8') as rf:
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([review, 1])
                elif label == 'neg':
                    data.append([review, 0])
    return data

train_data = readIMDB('E:\\语料\\aclImdb_v1\\aclImdb')
test_data = readIMDB('E:\\语料\\aclImdb_v1\\aclImdb', 'test')


In [3]:
def tokenizer(text):
    return [tok.lower() for tok in text.split(' ')]

train_tokenized = []
test_tokenized = []
for review, score in train_data:
    train_tokenized.append(tokenizer(review))
for review, score in test_data:
    test_tokenized.append(tokenizer(review))

vocab = set(chain(*train_tokenized))
vocab_size = len(vocab)


In [4]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('E:\\语料\word2vec\\glove.6B\\glove.6B.100d.txt',
                                                          binary=False, encoding='utf-8')
print(len(wvmodel.wv.vocab))

C:\softwareinstall\anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


391842


In [5]:
word_to_idx  = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

In [6]:
def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features

def pad_samples(features, maxlen=500, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features


In [8]:
train_features = torch.tensor(pad_samples(encode_samples(train_tokenized, vocab)))
train_labels = torch.tensor([score for _, score in train_data])
test_features = torch.tensor(pad_samples(encode_samples(test_tokenized, vocab)))
test_labels = torch.tensor([score for _, score in test_data])


In [19]:
class textCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len, labels, weight, **kwargs):
        super(textCNN, self).__init__(**kwargs)
        self.labels = labels
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.conv1 = nn.Conv2d(1, 1, (3, embed_size))
        self.conv2 = nn.Conv2d(1, 1, (4, embed_size))
        self.conv3 = nn.Conv2d(1, 1, (5, embed_size))
        self.pool1 = nn.MaxPool2d((seq_len - 3 + 1, 1))
        self.pool2 = nn.MaxPool2d((seq_len - 4 + 1, 1))
        self.pool3 = nn.MaxPool2d((seq_len - 5 + 1, 1))
        self.linear = nn.Linear(3, labels)

    def forward(self, inputs):
        inputs = self.embedding(inputs).view(inputs.shape[0], 1, inputs.shape[1], -1)
        x1 = F.relu(self.conv1(inputs))
        x2 = F.relu(self.conv2(inputs))
        x3 = F.relu(self.conv3(inputs))

        x1 = self.pool1(x1)
        x2 = self.pool2(x2)
        x3 = self.pool3(x3)

        x = torch.cat((x1, x2, x3), -1)
        x = x.view(inputs.shape[0], 1, -1)

        x = self.linear(x)
        x = x.view(-1, self.labels)

        return(x)

In [9]:

embed_size = 100
weight = torch.zeros(vocab_size+1, embed_size)

for i in range(len(wvmodel.index2word)):
    try:
        index = word_to_idx[wvmodel.index2word[i]]
    except:
        continue
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(
        idx_to_word[word_to_idx[wvmodel.index2word[i]]]))
    

In [10]:
wvmodel.get_vector('we')

array([-0.17791 ,  0.62675 ,  0.4787  , -0.55295 , -0.84935 , -0.070802,
       -0.34724 ,  0.4628  ,  0.12611 , -0.24875 ,  0.46881 ,  0.083636,
        0.56065 , -0.21931 ,  0.015561, -0.55806 , -0.20738 ,  0.9123  ,
       -1.2034  ,  0.30115 ,  0.46676 ,  0.483   , -0.10204 , -0.56799 ,
       -0.027126,  0.40567 , -0.14058 , -0.55485 ,  0.094588, -0.62213 ,
       -0.30343 ,  0.60639 ,  0.049799,  0.22204 ,  0.48549 ,  0.17629 ,
       -0.090535,  0.53705 ,  0.2755  , -0.78827 , -0.70953 , -0.16678 ,
        0.11206 , -0.48491 , -0.66644 ,  0.083952,  0.32885 , -0.45851 ,
       -0.37208 , -1.5315  ,  0.12994 , -0.2409  , -0.17219 ,  1.374   ,
       -0.22313 , -2.615   ,  0.35201 ,  0.33597 ,  1.6117  ,  0.92947 ,
       -0.37535 ,  0.82034 , -1.0677  , -0.45329 ,  1.2332  ,  0.23749 ,
        0.63523 ,  0.82859 , -0.1744  , -0.5853  ,  0.56339 , -0.73094 ,
        0.30815 , -1.0888  ,  0.46139 ,  0.045386, -0.17827 , -0.054054,
       -0.8831  ,  0.033935,  0.63083 , -0.19741 , 

In [20]:
num_epochs = 5
embed_size = 100
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 64
labels = 2
lr = 0.8
device = torch.device('cuda:0')
use_gpu = True

net = textCNN(vocab_size=(vocab_size+1), embed_size=embed_size,
              seq_len=500, labels=labels, weight=weight)

net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)


In [12]:
train_set = torch.utils.data.TensorDataset(train_features, train_labels)
test_set = torch.utils.data.TensorDataset(test_features, test_labels)

train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)


In [21]:
for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_losses = 0, 0
    train_acc, test_acc = 0, 0
    n, m = 0, 0
    for feature, label in train_iter:
        n += 1
        net.zero_grad()
        feature = Variable(feature.cuda())
        label = Variable(label.cuda())
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                 dim=1), label.cpu())
        train_loss += loss
    with torch.no_grad():
        for test_feature, test_label in test_iter:
            m += 1
            test_feature = test_feature.cuda()
            test_label = test_label.cuda()
            test_score = net(test_feature)
            test_loss = loss_function(test_score, test_label)
            test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                    dim=1), test_label.cpu())
            test_losses += test_loss
    end = time.time()
    runtime = end - start
    print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
          (epoch, train_loss.data / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))


epoch: 0, train loss: 0.6574, train acc: 0.62, test loss: 0.5692, test acc: 0.72, time: 6.82
epoch: 1, train loss: 0.6409, train acc: 0.63, test loss: 0.6475, test acc: 0.63, time: 6.43
epoch: 2, train loss: 0.5953, train acc: 0.69, test loss: 0.5950, test acc: 0.68, time: 5.25
epoch: 3, train loss: 0.6001, train acc: 0.69, test loss: 0.8209, test acc: 0.51, time: 5.24
epoch: 4, train loss: 0.5833, train acc: 0.70, test loss: 0.5518, test acc: 0.72, time: 5.23


In [ ]:
# reference:https://samaelchen.github.io/pytorch_lstm_sentiment/